In [ ]:
# Plot training loss and accuracy to evaluate model performance.

import matplotlib.pyplot as plt

# Plot loss
plt.plot(history.history["loss"], label="Loss")
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot accuracy
plt.plot(history.history["accuracy"], label="Accuracy")
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# Load a sample dataset, such as MovieLens, to build a recommendation system.

import pandas as pd

# Load the MovieLens dataset (replace with your dataset if needed)
url = "https://raw.githubusercontent.com/datasets/movielens-small/master/data/ratings.csv"
ratings = pd.read_csv(url)

print("Sample Data:")
print(ratings.head())

# Dataset columns:
# userId: unique identifier for a user
# movieId: unique identifier for a movie
# rating: rating given by the user to a movie
# timestamp: when the rating was given

In [ ]:
# Pivot the dataset to create a user-item matrix for collaborative filtering.

# Create a user-item matrix
user_item_matrix = ratings.pivot_table(index="userId", columns="movieId", values="rating")

# Fill missing values with 0 (or NaN for sparse matrices)
user_item_matrix.fillna(0, inplace=True)

print("User-Item Matrix:")
print(user_item_matrix.head())

In [ ]:
# Compute user similarities using cosine similarity.

from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between users
user_sim_matrix = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

print("User Similarity Matrix:")
print(user_sim_df.head())

In [ ]:
# Recommend items for a user based on their most similar users.

def recommend_items(user_id, user_item_matrix, user_sim_df, num_recommendations=5):
    similar_users = user_sim_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user itself
    user_ratings = user_item_matrix.loc[user_id]

    # Weighted sum of ratings from similar users
    weighted_sum = user_item_matrix.loc[similar_users].T.dot(user_sim_df[user_id].iloc[1:])
    normalized_weight = user_sim_df[user_id].iloc[1:].sum()
    recommendations = (weighted_sum / normalized_weight).sort_values(ascending=False)

    # Filter out items already rated by the user
    recommendations = recommendations[user_ratings.isna()][:num_recommendations]
    return recommendations

# Example: Recommend items for user 1
recommended_items = recommend_items(1, user_item_matrix, user_sim_df, num_recommendations=5)
print("Recommended Items for User 1:")
print(recommended_items)

In [ ]:
# Recommend items for a user based on their most similar users.

def recommend_items(user_id, user_item_matrix, user_sim_df, num_recommendations=5):
    similar_users = user_sim_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user itself
    user_ratings = user_item_matrix.loc[user_id]

    # Weighted sum of ratings from similar users
    weighted_sum = user_item_matrix.loc[similar_users].T.dot(user_sim_df[user_id].iloc[1:])
    normalized_weight = user_sim_df[user_id].iloc[1:].sum()
    recommendations = (weighted_sum / normalized_weight).sort_values(ascending=False)

    # Filter out items already rated by the user
    recommendations = recommendations[user_ratings.isna()][:num_recommendations]
    return recommendations

# Example: Recommend items for user 1
recommended_items = recommend_items(1, user_item_matrix, user_sim_df, num_recommendations=5)
print("Recommended Items for User 1:")
print(recommended_items)

In [ ]:
# Compute item similarities and recommend items based on a user's previously rated items.

# Compute cosine similarity between items
item_sim_matrix = cosine_similarity(user_item_matrix.T)
item_sim_df = pd.DataFrame(item_sim_matrix, index=user_item_matrix.columns, columns=user_item_matrix.columns)

print("Item Similarity Matrix:")
print(item_sim_df.head())

# Recommend items for a user based on their rated items
def recommend_items_based_on_items(user_id, user_item_matrix, item_sim_df, num_recommendations=5):
    user_ratings = user_item_matrix.loc[user_id]
    user_rated_items = user_ratings[user_ratings > 0].index

    # Weighted sum of similarities for rated items
    scores = item_sim_df[user_rated_items].dot(user_ratings[user_rated_items])
    scores = scores[~scores.index.isin(user_rated_items)]  # Exclude already rated items
    recommendations = scores.sort_values(ascending=False)[:num_recommendations]
    return recommendations

# Example: Recommend items for user 1
item_based_recommendations = recommend_items_based_on_items(1, user_item_matrix, item_sim_df, num_recommendations=5)
print("Item-Based Recommendations for User 1:")
print(item_based_recommendations)

In [ ]:
# Evaluate the recommendation system using metrics like RMSE or precision/recall.

from sklearn.metrics import mean_squared_error

# Example: Compute RMSE for user-item matrix reconstruction
original_ratings = user_item_matrix.values
predicted_ratings = user_sim_matrix.dot(user_item_matrix) / user_sim_matrix.sum(axis=1)[:, None]
mask = original_ratings > 0  # Only evaluate for non-zero entries

rmse = np.sqrt(mean_squared_error(original_ratings[mask], predicted_ratings[mask]))
print("RMSE:", rmse)